In [1]:

import sys
sys.path.append("/Users/a642163/Projects/tf-transformers/src/")

#
# macOS Notice
#
# This file is not consulted for DNS hostname resolution, address
# resolution, or the DNS query routing mechanism used by most
# processes on this system.
#
# To view the DNS configuration used by this system, use:
#   scutil --dns
#
# SEE ALSO
#   dns-sd(1), scutil(8)
#
# This file is automatically generated.
#
search fmr.com fmrco.com im.fmrco.com fi.fmrco.com
nameserver 10.165.9.56
nameserver 10.168.208.20
cat: /Users/a642163/.wgetrcb: No such file or directory
10.239.228.20:8000
10.239.228.20:8000


In [ ]:
# This notebook is intented to test, some of the
# results validation of GPT2 model 



In [2]:
import tensorflow as tf
import numpy as np
from tf_transformers.models import  GPT2Model 
from transformers import GPT2Tokenizer

In [3]:
# Globals
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

### 1. Check TF Conversion

In [4]:
# Check TF conversion

# !rm -rf /tmp/tf_transformers_cache/t5-base

model = GPT2Model.from_pretrained(model_name=model_name, convert_fn_type='tf')

You are using a model of type gpt2 to instantiate a model of type . This is not supported for all configurations of models and can yield errors.


Downloading:   0%|          | 0.00/498M [00:00<?, ?B/s]

INFO:absl:Successful ✅: Converted model using TF HF
INFO:absl:Successful: Saved model at /var/folders/9v/54jzg8fs4v91m_50ydft_1yx7cmk7x/T/tf_transformers_cache/gpt2/ckpt-1
INFO:absl:Successful ✅: Asserted and Converted `gpt2` from HF and saved it in cache folder /var/folders/9v/54jzg8fs4v91m_50ydft_1yx7cmk7x/T/tf_transformers_cache/gpt2


### 2.Check PT Conversion

In [5]:
# Check PT conversion

# !rm -rf /tmp/tf_transformers_cache/t5-base

model = GPT2Model.from_pretrained(model_name=model_name, convert_fn_type='pt')

INFO:absl:Successful: Model checkpoints matched and loaded from /var/folders/9v/54jzg8fs4v91m_50ydft_1yx7cmk7x/T/tf_transformers_cache/gpt2/ckpt-1


In [5]:
# Load model auto regressive
model_ar, config = GPT2Model.from_pretrained(model_name=model_name,
                                       use_auto_regressive = True,
                                       return_config=True)

You are using a model of type gpt2 to instantiate a model of type . This is not supported for all configurations of models and can yield errors.



Two checkpoint references resolved to different objects (<tf_transformers.models.gpt2.gpt2.GPT2Encoder object at 0x7f21cf27f6d0> and <keras.engine.input_layer.InputLayer object at 0x7f21cc4700a0>).



Two checkpoint references resolved to different objects (<tf_transformers.models.gpt2.gpt2.GPT2Encoder object at 0x7f21cf27f6d0> and <keras.engine.input_layer.InputLayer object at 0x7f21cc4700a0>).
INFO:absl:Successful: Model checkpoints matched and loaded from /tmp/tf_transformers_cache/gpt2


### 3. Test GPT2 with and without caching (Greedy)

In [12]:
# T5 text generation without caching
text = "I would love to walk with my cat because"

# Create inputs
inputs_hf = tokenizer(text, return_tensors='tf')
inputs = {}
inputs['input_ids'] = inputs_hf['input_ids']

# Iterate
predictions_non_auto_regressive = []
predictions_prob_non_auto_regressive = []

for i in range(20):
    outputs = model(inputs)
    predicted_ids = tf.cast(tf.expand_dims(tf.argmax(outputs["last_token_logits"], axis=1), 1), tf.int32)
    inputs["input_ids"] = tf.concat([inputs["input_ids"], predicted_ids], axis=1)
    predictions_non_auto_regressive.append(predicted_ids)
    predictions_prob_non_auto_regressive.append(
        tf.expand_dims(tf.reduce_max(outputs["last_token_logits"], axis=1), 1)
    )
predictions_non_auto_regressive = tf.concat(predictions_non_auto_regressive, axis=1)
predictions_prob_non_auto_regressive = tf.concat(predictions_prob_non_auto_regressive, axis=1)

#-------------------------------------------------------------------------------------------------------------#
# Text generation with cache
encoder_hidden_dim = config['embedding_size']
num_hidden_layers  = config['num_hidden_layers']
num_attention_heads = config['num_attention_heads']
attention_head_size = config['attention_head_size']

# Inputs
inputs_hf = tokenizer(text, return_tensors='tf')
input_ids = inputs_hf['input_ids']

batch_size = tf.shape(input_ids)[0]
seq_length = tf.shape(input_ids)[1]

all_cache_key = tf.zeros((num_hidden_layers, 
                                  batch_size, 
                                  num_attention_heads, 
                                  seq_length, 
                                  attention_head_size))
all_cahce_value = tf.zeros((num_hidden_layers, 
                                  batch_size, 
                                  num_attention_heads, 
                                  seq_length, 
                                  attention_head_size))


inputs = {}
inputs['input_ids'] = input_ids
inputs['all_cache_key'] = all_cache_key
inputs['all_cache_value'] = all_cahce_value
inputs["past_length"] = tf.zeros(shape=(1, batch_size), dtype=tf.int32)

# Iterate
predictions_auto_regressive = []
predictions_prob_auto_regressive = []
past_lengths = []
for i in range(20):
    outputs = model_ar(inputs)
    predicted_ids = tf.cast(tf.expand_dims(tf.argmax(outputs["last_token_logits"], axis=1), 1), tf.int32)
    
    # This is useful for variable batch decoding
    if i == 0:
        masks = tf.cast(tf.not_equal(input_ids, -1), tf.float32)
        masks = tf.reshape(
            masks,
            (1, batch_size, 1, seq_length, 1),
        )
        outputs["all_cache_key"]   = outputs["all_cache_key"]   * masks
        outputs["all_cache_value"] = outputs["all_cache_value"] * masks
        
    inputs["input_ids"] = predicted_ids
    inputs["all_cache_key"] = outputs["all_cache_key"]
    inputs["all_cache_value"] = outputs["all_cache_value"]
    inputs["past_length"] = outputs["past_length"]
    past_lengths.append(inputs["past_length"])
    predictions_auto_regressive.append(predicted_ids)
    predictions_prob_auto_regressive.append(
        tf.expand_dims(tf.reduce_max(outputs["last_token_logits"], axis=1), 1)
    )
predictions_auto_regressive = tf.concat(predictions_auto_regressive, axis=1)
predictions_prob_auto_regressive = tf.concat(predictions_prob_auto_regressive, axis=1)

#----------------------------------------------------------------------------------------#
expected_outputs = [[ 314, 1842,  607,  523,  881,   13,  314, 1842,  607,  523,  881,
          13,  314, 1842,  607,  523,  881,   13,  314, 1842]]
tf.assert_equal(predictions_non_auto_regressive, predictions_auto_regressive)
assert(np.allclose(predictions_prob_non_auto_regressive.numpy(), 
            predictions_prob_auto_regressive.numpy(),expected_outputs) == True)
print("Success")


### 4. Test GPT2 with TextDecoder Saved Model (Greedy)

In [15]:
# Text generation using saved_model with TextDecoder

# import tempfile
# import shutil
# from tf_transformers.text import TextDecoder
# text = ["I would love to walk with my cat because", 
#         "I love stars because"]

# # Save as saved model
# saved_model_dir = tempfile.mkdtemp()
# model_ar.save_as_serialize_module(saved_model_dir, overwrite=True)

# Load saved model
loaded   = tf.saved_model.load(saved_model_dir)
decoder  = TextDecoder(
    model = loaded
)

# Inputs
# Batching using -1 is must
input_ids = tf.ragged.constant(tokenizer(text)['input_ids']).to_tensor(-1)
inputs = {}
inputs['input_ids'] = input_ids

decoder_results = decoder.decode(inputs, 
               mode='greedy', 
               max_iterations=13, 
               eos_id=1)

expected_outputs = [[[314, 1842, 607, 523, 881, 13, 314, 1842, 607, 523, 881, 13, 314]],
 [[484, 821, 523, 881, 517, 621, 655, 257, 3491, 13, 1119, 821, 257]]]
assert(decoder_results['predicted_ids'].numpy().tolist()[0] == expected_outputs)

### 5. Test GPT2 with TextDecoder Saved Model (Beam)

In [21]:
# Beam check
decoder_results = decoder.decode(inputs, 
               mode='beam',
               num_beams=3,
               max_iterations=13, 
               eos_id=-100)
top_prediction = decoder_results['predicted_ids'].numpy().tolist()[0][0]
assert([top_prediction] == expected_outputs[0])

### 6. Test GPT2 with TextDecoder Saved Model (Top K top P)

In [25]:
decoder_results = decoder.decode(inputs, 
               mode='top_k_top_p',
               num_return_sequences=1,
                                 top_k=100,
                                 top_p=0.6,
               max_iterations=13, 
               eos_id=-100)

### 7. Test GPT2 with TextDecoderSerializable (Greedy)


In [40]:
# Text generation using saved_model with TextDecoderSerializable

import tempfile
import shutil
from tf_transformers.text import TextDecoderSerializable

decoder  = TextDecoderSerializable(
    model = model_ar,
    max_iterations=20,
    mode="greedy",
    do_sample=False,
    eos_id=-100,
)

# Save
decoder_model = decoder.get_model()
decoder_model.save_serialized(saved_model_dir, overwrite=True)

# Load
loaded_decoder   = tf.saved_model.load(saved_model_dir)
model_pb_decoder = loaded_decoder.signatures['serving_default']

text = "I would love to walk with my cat because"
inputs_hf = tokenizer(text, return_tensors='tf')
inputs = {}
inputs['input_ids'] = inputs_hf['input_ids']


decoder_results_serialized = model_pb_decoder(**inputs)
expected_outputs = [[ 314, 1842,  607,  523,  881,   13,  314, 1842,  607,  523,  881,
          13,  314, 1842,  607,  523,  881,   13,  314, 1842]]
assert(decoder_results_serialized['predicted_ids'].numpy().tolist()[0] == expected_outputs)

INFO:tensorflow:Assets written to: /tmp/tmp0cvx8lmc/assets


INFO:tensorflow:Assets written to: /tmp/tmp0cvx8lmc/assets


### 8. Test GPT2 with TextDecoderSerializable (Beam)


In [ ]:
# Text generation using saved_model with TextDecoderSerializable

import tempfile
import shutil
from tf_transformers.text import TextDecoderSerializable

decoder  = TextDecoderSerializable(
    model = model_ar,
    max_iterations=20,
    num_beams=3,
    mode="beam",
    do_sample=False
)

# Save
decoder_model = decoder.get_model()
decoder_model.save_serialized(saved_model_dir, overwrite=True)

# Load
loaded_decoder   = tf.saved_model.load(saved_model_dir)
model_pb_decoder = loaded_decoder.signatures['serving_default']

text = "I would love to walk with my cat because"
inputs_hf = tokenizer(text, return_tensors='tf')
inputs = {}
inputs['input_ids'] = inputs_hf['input_ids']


decoder_results_serialized = model_pb_decoder(**inputs)
top_prediction = decoder_results_serialized['predicted_ids'].numpy().tolist()[0][0]
assert([top_prediction] == expected_outputs)


### 9. Test GPT2 with TextDecoderSerializable (Top K top P)


In [ ]:
import tempfile
import shutil
from tf_transformers.text import TextDecoderSerializable

# loaded   = tf.saved_model.load(saved_model_dir)
decoder  = TextDecoderSerializable(
    model = model_ar,
    max_iterations=20,
    top_k=100,
    top_p=0.7,
    mode="top_k_top_p",
    do_sample=False,
)

# Save
decoder_model = decoder.get_model()
decoder_model.save_serialized(saved_model_dir, overwrite=True)

# Load
loaded_decoder   = tf.saved_model.load(saved_model_dir)
model_pb_decoder = loaded_decoder.signatures['serving_default']

text = "I would love to walk with my cat because"
inputs_hf = tokenizer(text, return_tensors='tf')
inputs = {}
inputs['input_ids'] = inputs_hf['input_ids']


decoder_results_serialized = model_pb_decoder(**inputs)

### 10. Test GPT2 tf lite


In [ ]:
model = Model.from_pretrained(model_name=MODEL_NAME, batch_size=1, sequence_length=32,)

tempdir = tempfile.mkdtemp()
model.save_serialized(tempdir, overwrite=True)

converter = tf.lite.TFLiteConverter.from_saved_model("{}".format(tempdir))  # path to the SavedModel directory
converter.experimental_new_converter = True

tflite_model = converter.convert()
open("{}/converted_model.tflite".format(tempdir), "wb").write(tflite_model)

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="{}/converted_model.tflite".format(tempdir))
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Get result
# encoder input_ids
interpreter.set_tensor(
    input_details[0]['index'],
    tf.random.uniform(input_details[0]['shape'], minval=0, maxval=100, dtype=tf.int32),
)
interpreter.invoke()
tflite_output = interpreter.get_tensor(output_details[-1]['index'])
print("Tflite output shape", tflite_output.shape)
# tf.debugging.assert_equal(tflite_output.shape, (1, 32, 32128))
logging.info("Test: TFlite Conversion. ✅")
shutil.rmtree(tempdir)